In [ ]:
!pip install sweetviz
!pip install pandas
!pip install dataprofiler
!pip install ydata_profiling


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 79.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.7/738.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.6 MB/s eta 0:00:00
  Created wheel for HLL: filename=HLL-2.3.0-cp311-cp311-linux_x86_64.whl size=32970 sha256=e99acafa9fb80a3cc49737019950c4e45109fb9c90ff9237fd719de552fa6f4f
  Stored in directory: /root/.cache/pip/wheels/ab/5a/b5/65fdce434afc0cd43e14484d244c3928c7f1a7943dd2ab1a2c
Successfully built HLL
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install numpy==1.26.4 --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
from ydata_profiling import ProfileReport
import pandas as pd

df = pd.read_csv('/content/healthcare_dataset.csv')

profile = ProfileReport(df, title="Healthcare Data Profiling Report")
profile.to_file("ydata_report.html")


In [ ]:
!pip install pandera

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.2/264.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
df.isna().sum()

,0
Name,0
Age,0
Gender,0
Blood Type,0
Medical Condition,0
Date of Admission,0
Doctor,0
Hospital,0
Insurance Provider,0
Billing Amount,0


In [ ]:
import pandas as pd
import pandera as pa
from pandera import Column, DataFrameSchema, Check

df = pd.read_csv('/content/healthcare_dataset.csv')


df = df.drop_duplicates()


df['Date of Admission'] = pd.to_datetime(df['Date of Admission'], errors='coerce')
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'], errors='coerce')


text_cols = ['Name', 'Gender', 'Medical Condition', 'Doctor',
             'Insurance Provider', 'Admission Type', 'Medication', 'Test Results']
for col in text_cols:
    df[col] = df[col].str.strip().str.title()



df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Billing Amount'] = pd.to_numeric(df['Billing Amount'], errors='coerce').abs()
df['Room Number'] = pd.to_numeric(df['Room Number'], errors='coerce')


cat_cols = ['Gender', 'Blood Type', 'Medical Condition', 'Doctor', 'Hospital',
            'Insurance Provider', 'Admission Type', 'Medication', 'Test Results']
for col in cat_cols:
    df[col] = df[col].astype('category')



df.to_csv('/content/healthcare_dataset_mylast_cleaned.csv', index=False)
print("✅ Dataset cleaned and saved!")







schema = pa.DataFrameSchema({
    "Name": Column(str, nullable=False),
    "Age": Column(int, Check.in_range(0, 120), nullable=False),
    "Gender": Column(pa.Category, Check.isin(["Male", "Female"]), nullable=False),
    "Blood Type": Column(pa.Category, Check.isin(["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"]), nullable=False),
    "Medical Condition": Column(pa.Category, nullable=False),
    "Date of Admission": Column(pa.DateTime, nullable=False),
    "Doctor": Column(pa.Category, nullable=False),
    "Hospital": Column(pa.Category, nullable=False),
    "Insurance Provider": Column(pa.Category, nullable=False),
    "Billing Amount": Column(float, Check.ge(0), nullable=False),
    "Room Number": Column(int, Check.ge(0), nullable=False),
    "Admission Type": Column(pa.Category, Check.isin(["Emergency", "Elective", "Urgent"]), nullable=False),
    "Discharge Date": Column(pa.DateTime, nullable=False),
    "Medication": Column(pa.Category, nullable=False),
    "Test Results": Column(pa.Category, Check.isin(["Normal", "Abnormal", "Inconclusive"]), nullable=False),
})



try:
    validated_df = schema.validate(df, lazy=True)
    print("✅ All rows are valid and schema-compliant!")
    print(validated_df.head())
except pa.errors.SchemaErrors as e:
    print("❌ Validation errors found:")
    print(e.failure_cases)







validated_df.info()

<ipython-input-4-721c3e23cbf5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date of Admission'] = pd.to_datetime(df['Date of Admission'], errors='coerce')
<ipython-input-4-721c3e23cbf5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Discharge Date'] = pd.to_datetime(df['Discharge Date'], errors='coerce')
<ipython-input-4-721c3e23cbf5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

✅ Dataset cleaned and saved!


/usr/local/lib/python3.11/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


✅ All rows are valid and schema-compliant!
            Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby Jackson   30    Male         B-            Cancer        2024-01-31   
1   Leslie Terry   62    Male         A+           Obesity        2019-08-20   
2    Danny Smith   76  Female         A-           Obesity        2022-09-22   
3   Andrew Watts   28  Female         O+          Diabetes        2020-11-18   
4  Adrienne Bell   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell                    Cook PLC              Aetna   
3       Kevin Wells  Hernandez Rogers and Vang,           Medicare   
4    Kathleen Hanna                 White-White              Aetna   

   Billing Amount  Room Number Admission Type Discharge

In [ ]:
import hashlib


def hash_data(data: str) -> str:
    return hashlib.sha256(data.encode()).hexdigest()



def caesar_encrypt(text: str, shift: int = 3) -> str:
    encrypted = ""
    for char in text:
        if char.isalpha():
            base = 65 if char.isupper() else 97
            encrypted += chr((ord(char) - base + shift) % 26 + base)
        else:
            encrypted += char
    return encrypted

def caesar_decrypt(text: str, shift: int = 3) -> str:
    return caesar_encrypt(text, -shift)



class User:
    def __init__(self, username: str, role: str):
        self.username = username
        self.role = role

    def can_access(self, data_type: str) -> bool:
        permissions = {
            "admin": ["all"],
            "analyst": ["read", "write"],
            "viewer": ["read"]
        }
        allowed = permissions.get(self.role, [])
        return "all" in allowed or data_type in allowed




class GDPRHandler:
    def request_consent(self, user_id):
        print(f"Consent requested from user {user_id} under GDPR.")

    def erase_user_data(self, user_id):
        print(f"Data for user {user_id} erased as per GDPR.")

class CCPAHandler:
    def process_access_request(self, email):
        print(f"CCPA access request processed for {email}.")

class HIPAAHandler:
    def scan_system(self):
        print("Running HIPAA compliance scan using pyTenable (mock).")



if __name__ == "__main__":
    # Hashing
    print("Hashed Password:", hash_data("MySecurePassword123"))
    print("Hashed Phone Number:", hash_data("123-456-7890"))

    # Caesar Cipher
    secret = "Confidential Notes"
    encrypted = caesar_encrypt(secret)
    decrypted = caesar_decrypt(encrypted)
    print(f"\nEncrypted: {encrypted}")
    print(f"Decrypted: {decrypted}")

    # RBAC
    user = User("jane_doe", "analyst")
    print(f"\nCan 'analyst' write?: {user.can_access('write')}")

    # GDPR
    gdpr = GDPRHandler()
    gdpr.request_consent("user123")
    gdpr.erase_user_data("user123")

    # CCPA
    ccpa = CCPAHandler()
    ccpa.process_access_request("user@example.com")

    # HIPAA
    hipaa = HIPAAHandler()
    hipaa.scan_system()


Hashed Password: ccf9ac1c9ce02b9bb7810a1fff51e474f37d98c3582f2d3b5036caf559afd9bc
Hashed Phone Number: 29ec0a06044bedff0bec4d81516da5843c93b7b4cb8b3ec0bf379a78085747bd

Encrypted: Frqilghqwldo Qrwhv
Decrypted: Confidential Notes

Can 'analyst' write?: True
Consent requested from user user123 under GDPR.
Data for user user123 erased as per GDPR.
CCPA access request processed for user@example.com.
Running HIPAA compliance scan using pyTenable (mock).


In [ ]:
from collections import Counter

def caesar_encrypt(text, shift):
    result = ''
    for char in text:
        if char.isalpha():
            base = ord('A') if char.isupper() else ord('a')
            result += chr((ord(char) - base + shift) % 26 + base)
        else:
            result += char
    return result

def caesar_decrypt(cipher_text, shift):
    return caesar_encrypt(cipher_text, -shift)

def frequency_analysis_attack(cipher_text):
    # Get frequency of letters
    filtered_text = ''.join(filter(str.isalpha, cipher_text.lower()))
    frequency = Counter(filtered_text)
    most_common_letter = frequency.most_common(1)[0][0]
    # Assume the most common letter should be 'e'
    assumed_shift = (ord(most_common_letter) - ord('e')) % 26
    return caesar_decrypt(cipher_text, assumed_shift), assumed_shift

# Test code
original_text = "This is a top secret message. Do not share."
cipher_text = caesar_encrypt(original_text, 3)
decrypted_guess, guessed_shift = frequency_analysis_attack(cipher_text)

print("Original Encrypted:", cipher_text)
print("Guessed Shift:", guessed_shift)
print("Decrypted by Frequency Analysis:", decrypted_guess)
print("Matches Original:", decrypted_guess.lower() == original_text.lower())


Original Encrypted: Wklv lv d wrs vhfuhw phvvdjh. Gr qrw vkduh.
Guessed Shift: 17
Decrypted by Frequency Analysis: Ftue ue m fab eqodqf yqeemsq. Pa zaf etmdq.
Matches Original: False
